<a href="https://colab.research.google.com/github/toshiikuoo/puclic/blob/master/%E6%A0%AA%E4%BE%A1%E7%9B%B8%E9%96%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#必要なライブラリimport
import pandas as pd
from pandas import Series,DataFrame
from pandas_datareader import DataReader

import numpy as np

from datetime import datetime

import matplotlib as mat
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('whitegrid')

from scipy.stats.stats import pearsonr
import itertools



In [0]:
#S&P全銘柄のリストを作成
url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_list=pd.io.html.read_html(url)

In [0]:
sp500_list_first=sp500_list[0]

In [0]:
#銘柄シンボルのみを取り出したSeriesを作成
sp500_symbol=sp500_list_first["Symbol"]

In [0]:
#listに変換
sp500_symbol_list=sp500_symbol.values.tolist()

In [0]:
#yahooでは銘柄シンボルの"."は"-"に変えないとエラーになるので置換
#"."を含む要素を確認
sp500_symbol_list_in=[i for i in sp500_symbol_list if "." in i]

In [0]:
#"."を"-"に置換
sp500_symbol_list_yahoo=[i.replace(".","-")for i in sp500_symbol_list]

In [8]:
#確認のため"-"の含まれる要素を表示"
sp500_symbol_list_yahoo_in=[i for i in sp500_symbol_list_yahoo if "-" in i]
print(sp500_symbol_list_yahoo_in)

['BRK-B', 'BF-B']


In [0]:
#sp500銘柄の終値を１つのDataFrameに格納
close_sp500_list=DataReader(sp500_symbol_list_yahoo,'yahoo',datetime(2019, 9, 26, 7, 10, 37, 334315),datetime(2019, 10, 26, 7, 10, 37, 334315),)["Adj Close"]

In [0]:
#株のシンボルのlistを作成
columns=close_sp500_list.columns.to_list()

In [0]:
#すべての株のペアの組み合わせを出力
print(list(itertools.combinations(columns,2)))

In [0]:
correlations={}

In [0]:
#列ごとのペアで相関を算出
for cola,colb in itertools.combinations(columns,2):
  nas=np.logical_or(np.isnan(close_sp500_list.loc[:,cola]),np.isnan(close_sp500_list.loc[:,colb]))
  correlations[cola + '__'+ colb]=pearsonr(close_sp500_list.loc[:,cola][~nas],close_sp500_list.loc[:,colb][~nas])

In [0]:
#出力された結果"correlations"はリスト形式なのでDataFrameに変換
result=DataFrame.from_dict(correlations,orient='index')
result.columns=['PCC','p-value']
print(result.sort_values('PCC'))